In [6]:
from mtg_draftbot.draftbot.draftbot_sim import *
from draftbot_sim_adapted import MultiStratDraft
import sqlite3
import pandas as pd
import numpy as np
import json
import os

In [10]:
draft_path = 'C:/Users/trist/mtg_research/test_draft.txt'
cards_path = 'C:/Users/trist/mtg_research/mtg_draftbot/data/m20/m20-cards.json'
weights_path = 'C:/Users/trist/mtg_research/mtg_draftbot/data/new_generated_weights.json'

In [8]:
def comp_generator(num_iter,n_drafters_strat_one,n_drafters_strat_two,n_cards_in_pack,rounds,cards_path,card_values_path,packs_input_file):
    '''Streamline comparison of drafts and create outputs with player prefs;
    this gives us quick glances into proxies for how '''

    n_drafters = n_drafters_strat_one + n_drafters_strat_two
    ss_prefs = []
    ms_prefs = []

    r = 1

    for r in range(1,num_iter+1):
          draft_ss = Draft(n_drafters=n_drafters,
                    n_rounds=rounds,
                    n_cards_in_pack=n_cards_in_pack,
                    cards_path=cards_path,
                    card_values_path=card_values_path)

    draft_ms = MultiStratDraft(n_drafters_strat_one=n_drafters_strat_one,
                    n_drafters_strat_two=n_drafters_strat_two,
                    n_rounds=rounds,
                    n_cards_in_pack=n_cards_in_pack,
                    cards_path=cards_path,
                    card_values_path=card_values_path,
                    packs_input_file=packs_input_file
                    )

    draft_ss.draft()
    result_array = np.empty(0)
     #result_array = np.append(result_array,np.arange(0,draft_ss.n_drafters))
    for item,idx in enumerate(draft_ss.drafter_preferences):
          player=np.append(item,r)
          player=np.append(player,idx)
          result_array = np.append(result_array,player)
    ss_prefs.append(result_array.reshape((8,12)))

    if os.path.exists(packs_input_file):
               os.remove(packs_input_file)
    save_packs_array(draft_ss,packs_input_file)

    draft_ms.draft()
    result_array = np.empty(0)
    for item,idx in enumerate(draft_ms.drafter_preferences):
          player=np.append(item,r)
          player=np.append(player,idx)
          result_array = np.append(result_array,player)

    ms_prefs.append(result_array.reshape((8,12)))
    r += 1


    return (ms_prefs,ss_prefs)

def macro_prefs_sum_stats(draft_comp):
    '''Take a draft comp and output results'''

    ss_prefs = draft_comp[1]
    ms_prefs = draft_comp[0]
    ss_prefs_df = pd.DataFrame(np.concatenate(ss_prefs))
    ms_prefs_df = pd.DataFrame(np.concatenate(ms_prefs))

    ssdf = ss_prefs_df.set_index([0,1])
    ssdf = ssdf.groupby(by=[0]).mean()
    ssdf['maximum'] = ssdf.max(axis=1)
    ssdf['minimum'] = ssdf.min(axis=1)
    ssdf['average'] = ssdf.mean(axis=1)
    ssdf['stdev'] = ssdf.std(axis=1)

    msdf = ms_prefs_df.set_index([0,1])
    msdf = msdf.groupby(by=[0]).mean()
    msdf['maximum'] = msdf.max(axis=1)
    msdf['minimum'] = msdf.min(axis=1)
    msdf['average'] = msdf.mean(axis=1)
    msdf['stdev'] = msdf.std(axis=1)

    return (msdf,ssdf)

In [11]:
one_k_comp = comp_generator(1000,1,7,14,3,cards_path,weights_path,
draft_path)

one_k_twosix = comp_generator(1000,2,6,14,3,cards_path,weights_path,
draft_path)

threefive_k = comp_generator(1000,3,5,14,3,cards_path,weights_path,
draft_path)

fourfour_k = comp_generator(1000,4,4,14,3,cards_path,weights_path,
draft_path)

In [37]:
thousand_one = macro_prefs_sum_stats(one_k_comp)
thousand_two = macro_prefs_sum_stats(one_k_twosix)
thousand_three = macro_prefs_sum_stats(threefive_k)
thousand_four = macro_prefs_sum_stats(fourfour_k)

In [41]:
#Difference in maxes is largest for the position with the new strategy
#Note that a negative reduction here would be analogous to saying that a preference toward the most preferred archetype goes down
#That really means that drafter 0 has a more diverse/less specialized pool because they don't have as strong a favored archetype as the others. 

print(thousand_one[0]['average'] - thousand_one[1]['average'])
print(thousand_two[0]['average'] - thousand_two[1]['average'])

0
0.0   -0.371814
1.0    0.286203
2.0   -0.073507
3.0   -0.200329
4.0   -0.081856
5.0   -0.266641
6.0    0.005531
7.0   -0.005342
Name: average, dtype: float64
0
0.0   -1.034843
1.0   -0.273083
2.0    0.447033
3.0    0.131583
4.0    0.083300
5.0   -0.068632
6.0    0.113245
7.0    0.017839
Name: average, dtype: float64


In [40]:
print(thousand_three[0]['average'] - thousand_three[1]['average'])
print(thousand_four[0]['average'] - thousand_four[1]['average'])

0
0.0   -0.287395
1.0   -0.348232
2.0   -0.855627
3.0    0.548763
4.0    0.234865
5.0    0.426876
6.0    0.084884
7.0    0.034544
Name: average, dtype: float64
0
0.0   -0.391814
1.0   -0.741677
2.0   -0.064369
3.0    0.158388
4.0    0.904265
5.0    0.646894
6.0   -0.012300
7.0   -0.060655
Name: average, dtype: float64
